In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Random import get_random_bytes
import base64

# Generate RSA key pair
def generate_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

# Encrypt message with AES, and RSA for AES key
def encrypt_message(message, public_key):
    aes_key = get_random_bytes(16)
    cipher_aes = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(message.encode())
    
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    encrypted_key = cipher_rsa.encrypt(aes_key)

    return base64.b64encode(encrypted_key + cipher_aes.nonce + tag + ciphertext)

# Decrypt message using RSA and AES
def decrypt_message(encrypted_message, private_key):
    data = base64.b64decode(encrypted_message)
    encrypted_key = data[:256]
    nonce = data[256:272]
    tag = data[272:288]
    ciphertext = data[288:]
    
    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    aes_key = cipher_rsa.decrypt(encrypted_key)
    
    cipher_aes = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
    return cipher_aes.decrypt_and_verify(ciphertext, tag).decode()

# Example usage
private_key, public_key = generate_keys()
message = "Hello, secure world!"
encrypted_msg = encrypt_message(message, public_key)
decrypted_msg = decrypt_message(encrypted_msg, private_key)
print("Original:", message)
print("Decrypted:", decrypted_msg)